In [1]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns

In [2]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22000 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Tue Aug 09 12:41:10 2022 Hora oficial do Brasil

In [3]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
# local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [4]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((49, 60*fs-30))
COPYreal = np.zeros((49, 60*fs-30))
COPZreal = np.zeros((49, 60*fs-30))

d1x = np.zeros((49, 60*fs-30))
d2x = np.zeros((49, 60*fs-30))

d1z = np.zeros((49, 60*fs-30))
d2z = np.zeros((49, 60*fs-30))

matrizangx = np.zeros((49, 60*fs-30))
matrizangz = np.zeros((49, 60*fs-30))

alpha = np.zeros((49, 60*fs-30))

COGX = np.zeros((49, 60*fs-30))
COGY = np.zeros((49,60*fs-30))
COGZ = np.zeros((49, 60*fs-30))  

for s in range (1,50):
    a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
    fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
    grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
    mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
    COGX[s-1,:]=mkr['COG_X'].values[30:]
    COGY[s-1,:]=mkr['COG_Y'].values[30:]
    COGZ[s-1,:]=mkr['COG_Z'].values[30:]
    #print(COGX)
    mass=PDSinfo['Mass']
    d = np.sqrt(COGX[s-1,:]**2+COGY[s-1,:]**2+COGZ[s-1,:]**2)
    angulox = np.arcsin(COGX[s-1,:]/d)
    anguloz= np.arcsin(COGZ[s-1,:]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[s-1,:] = angulox
    matrizangz[s-1,:] = anguloz
    
    
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[s-1,:]=np.gradient(matrizangx[s-1,:],dt)
    d1z[s-1,:]=np.gradient(matrizangz[s-1,:],dt)

    d2x[s-1,:]=np.gradient(d1x[s-1,:],dt)
    d2z[s-1,:]=np.gradient(d1z[s-1,:],dt)
    
    COPX[s-1,:] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[s-1,:]**2-cosx*d2x[s-1,:])
    COPZ[s-1,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[s-1,:]**2-cosz*d2z[s-1,:])
    
    
    COPXreal[s-1,:]=grf['COPNET_X'].values[30:]
    COPZreal[s-1,:]=grf['COPNET_Z'].values[30:]
    

    
    x= np.vstack((COPX[[s-1],:],d2x[[s-1],:]))
    y= COPXreal[s-1,:]

    model=LinearRegression(fit_intercept=False)
    model.fit(x.T,y)
    
    
    split=train_test_split(COGX,COPXreal,test_size=9)
    COGXtrain = split[0]
    COPXtrain = split[1]
    COGXtest = split[2]
    COPXtest = split[3]
    
#print(COGXtrain)
#for i in COGXtrain:
 #   i=0
#print(COGXtrain[i])
    
    
    
    
    #print(model.coef_) 
    
    #def reg_m(y, x):
        #ones = np.ones(len(x[0]))
        #X = sm.add_constant(np.column_stack((x[0], ones)))
        #for oi in x[1:]:
            #X = sm.add_constant(np.column_stack((oi, X)))
            #results = sm.OLS(y, x).fit()
            #return results
    #print(model.coef_)
    #print(reg_m(y, x.T).summary())
 
    #print(model.coef_)
    #x= (np.array([COPX[s-1,:],d2x[s-1,:]]))
    #y= COPXreal[s-1,:]
    #model=LinearRegression()
    #model.fit(x.T,y)
    #print(model.coef_) 
    #Histogramas
    #plt.figure()
    #hist=plt.hist(model.coef_[0], bins=50)
    #plt.title('Histograms of alpha'.format(s))
    #plt.show()


In [6]:
from sklearn.linear_model import LinearRegression
fs = 100 # Hz
COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((49, 60*fs-30))
COPYreal = np.zeros((49, 60*fs-30))
COPZreal = np.zeros((49, 60*fs-30))

d1x = np.zeros((49, 60*fs-30))
d2x = np.zeros((49, 60*fs-30))

d1z = np.zeros((49, 60*fs-30))
d2z = np.zeros((49, 60*fs-30))

matrizangx = np.zeros((49, 60*fs-30))
matrizangz = np.zeros((49, 60*fs-30))

alpha = np.zeros((49, 60*fs-30))

COGX = np.zeros((49, 60*fs-30))
COGY = np.zeros((49,60*fs-30))
COGZ = np.zeros((49, 60*fs-30)) 

COGXtrain = np.zeros((40, 60*fs-30))
COGYtrain = np.zeros((40,60*fs-30))
COGZtrain = np.zeros((40, 60*fs-30)) 

subject = np.zeros((0,1))
j=0

for s in range (1,50):
    a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    for i in range(len(a)):
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        if not np.any(np.isnan(mkr['COG_X'].values[30:])):
            COGX = np.vstack((COGX, mkr['COG_X'].values[30:]))
            COGY = np.vstack((COGY, mkr['COG_Y'].values[30:]))
            COGZ = np.vstack((COGZ, mkr['COG_Z'].values[30:]))
    
            mass=PDSinfo['Mass']
            d = np.sqrt(COGX[s-1,:]**2+COGY[s-1,:]**2+COGZ[s-1,:]**2)
            angulox = np.arcsin(COGX[s-1,:]/d)
            anguloz= np.arcsin(COGZ[s-1,:]/d)
            sinx=np.sin(angulox)
            sinz=np.sin(anguloz)
            cosx=np.cos(angulox)
            cosz=np.cos(anguloz)
            tanx=np.tan(angulox)
            tanz=np.tan(anguloz)
            matrizangx = np.vstack((matrizangx, angulox))
            matrizangz = np.vstack((matrizangz, anguloz))
    
    
            Jb=(4/3)*(mass.values[s])*((d)**2)
            t=grf['Time']
            dt=t[1]-t[0]

            d1x = np.vstack((d1x, np.gradient(matrizangx[s-1,:],dt)))
            d1z = np.vstack((d1z, np.gradient(matrizangz[s-1,:],dt)))

            d2x = np.vstack((d2x, np.gradient(d1x[s-1,:],dt)))
            d2z = np.vstack((d2z, np.gradient(d1z[s-1,:],dt)))

            COPX = np.vstack((COPX, d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[s-1,:]**2-cosx*d2x[s-1,:])))
            COPZ = np.vstack((COPZ, d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[s-1,:]**2-cosz*d2z[s-1,:])))


            COPXreal = np.vstack((COPXreal, grf['COPNET_X'].values[30:]))
            COPZreal = np.vstack((COPZreal, grf['COPNET_Z'].values[30:]))
            
            subject = np.vstack((subject, [s]))



TypeError: object of type 'int' has no len()

In [ ]:
splitX=train_test_split(COGX,COPXreal,COGY,COPYreal,COGZ,COPZreal,test_size = 9)
COGXtrain = splitX[0]
COGXtest = splitX[1]
COPXtrain = splitX[2]
COPXtest = splitX[3]
COGYtrain = splitX[4]
COGYtest = splitX[5]
COPYtrain = splitX[6]
COPYtest = splitX[7]
COGZtrain = splitX[8]
COGZtest = splitX[9]
COPZtrain = splitX[10]
COPZtest = splitX[11]

print(COGXtest.shape)

In [ ]:
#splitY=train_test_split(COGY,COPYreal,test_size=9)
#COGYtrain = splitY[0]
#COPYtrain = splitY[1]
#COGYtest = splitY[2]
#COPYtest = splitY[3]
#print(COGYtrain.shape)

In [ ]:
#splitZ=train_test_split(COGZ,COPZreal,test_size=9)
#COGZtrain = splitZ[0]
#COPZtrain = splitZ[1]
#COGZtest = splitZ[2]
#COPZtest = splitZ[3]
#print(COGZtrain.shape)

In [ ]:
from sklearn.model_selection import train_test_split
alpha= []
beta=[]
for i in range(1,40):
    d = np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2)
    angulox = np.arcsin(COGXtrain[i]/d)
    anguloz = np.arcsin(COGZtrain[i]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[i,:] = angulox
    matrizangz[i,:] = anguloz
    
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[i,:]=np.gradient(matrizangx[i,:],dt)
    d1z[i,:]=np.gradient(matrizangz[i,:],dt)
    
    

    d2x[i,:]=np.gradient(d1x[i,:],dt)
    d2z[i,:]=np.gradient(d1z[i,:],dt)
    
    COPX[i,:] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[i,:]**2-cosx*d2x[i,:])
    COPZ[i,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[i,:]**2-cosz*d2z[i,:])
    
    
    x= np.vstack((COPX[i,:],d2x[i,:]))
    y= COPXtrain[i,:]
    

    model=LinearRegression(fit_intercept=False)
    model.fit(x.T,y)
    
    alpha.append(model.coef_[0])
    
    beta.append(model.coef_[1])
   
alfamedia=np.mean(alpha)
betamedia=np.mean(beta)
print(alfamedia)
print(betamedia)

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
COPXES = np.zeros((9, 60*fs-30))
for u in range(0,9):
    d = np.sqrt(COGXtest[u]**2+COGYtest[u]**2+COGZtest[u]**2)
    angulox = np.arcsin(COGXtest[u]/d)
    anguloz = np.arcsin(COGZtest[u]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[u,:] = angulox
    matrizangz[u,:] = anguloz
    
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[u,:]=np.gradient(matrizangx[u,:],dt)
    d1z[u,:]=np.gradient(matrizangz[u,:],dt)
    
    

    d2x[u,:]=np.gradient(d1x[u,:],dt)
    d2z[u,:]=np.gradient(d1z[u,:],dt)
    
    
    COPX[u,:] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[u,:]**2-cosx*d2x[u,:])
    
    
    COPXES[u,:] = alfamedia* COPX[u,:] + betamedia*(d2x[u,:])
    
    
plt.figure()
plt.plot(COPXES[u,:],color='purple',label='COP calculated')
plt.plot(COPXtest[u],color='blue', label='COP real')
plt.title('COP real X COP calculated for tests')
plt.legend()
plt.show()
for u in range(1,9):
    plt.figure()
    plt.plot(COPXES[u,:],color='purple',label='COP calculated')
    plt.plot(COPXtest[u],color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(u))
    plt.legend()
    plt.show()